In [199]:
import json
import sqlite3
import glob
import os
import time

In [309]:
# Создаем соединение с базой данных
conn = sqlite3.connect('garpix_0.1.21.db')

In [310]:
# Создаем таблицы в базе данных
with open('garpix_0.1.10.sql', 'r') as sql_file:
    sql_script = sql_file.read()

cursor = conn.cursor()
cursor.executescript(sql_script)
conn.commit()

In [311]:
# Заполняем таблицу data данными из файла JSON
def load_data_table(data): 
    conn.execute("""
        INSERT INTO data (
            calculation_id,
            settings_rank,
            settings_max_step,
            settings_min_step,
            settings_usersort,
            settings_hardlimit,
            settings_timelimit,
            settings_step_scaling
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
        (
            int(data['data']['calculation_id']), 
            data['data']['settings']['rank'], 
            data['data']['settings']['max_step'],
            data['data']['settings']['min_step'],
            data['data']['settings']['usersort'], 
            data['data']['settings']['hardlimit'] if 'hardlimit' in data['data']['settings'] else None,
            data['data']['settings']['timelimit'] if 'timelimit' in data['data']['settings'] else None,
            data['data']['settings']['step_scaling']
        ))


In [312]:
# Заполняем таблицу data_cargoes данными из файла JSON
def load_data_cargoes_table(data): 
    for item in data['data']['cargoes']:
        conn.execute("""
            INSERT INTO data_cargoes (
                calculation_id  ,
                cargo_id        ,
                info_color      ,
                info_title      ,
                info_article    ,
                info_group_id   ,
                info_group_title,
                mass            ,
                size_width      ,
                size_height     ,
                size_length     ,
                sort            ,
                type            ,
                count           ,
                margin_width    ,
                margin_height   ,
                margin_length   ,
                stacking        ,
                turnover        ,
                stacking_limit  ,
                prepare_last_result
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                int(data['data']['calculation_id']), 
                item['cargo_id'],
                item['info']['color'] if 'color' in item['info'] else None,
                item['info']['title'] if 'title' in item['info'] else None,
                item['info']['article'] if 'article' in item['info'] else None,
                item['info']['group_id'] if 'group_id' in item['info'] else None,
                item['info']['group_title'] if 'group_title' in item['info'] else None,
                item['mass'],
                item['size']['width'], 
                item['size']['height'],
                item['size']['length'],
                item['sort'], 
                item['type'],
                item['count'],
                item['margin']['width'],
                item['margin']['height'], 
                item['margin']['length'],
                item['stacking'],
                item['turnover'], 
                item['stacking_limit'],
                item['prepare_last_result'] if 'prepare_last_result' in item else None,
            ))
    

In [313]:
# Заполняем таблицу data_cargo_space данными из файла JSON
def load_data_cargo_space_table(data): 
        conn.execute("""
            INSERT INTO data_cargo_space (
                id                   ,
                info_uid             ,
                info_sort            ,
                info_title           ,
                info_margin_width    ,
                info_margin_height   ,
                info_margin_length   ,
                mass                 ,
                size_width           ,
                size_height          ,
                size_length          ,
                type                 ,
                axles                ,
                title                ,
                margin_width              ,
                margin_height             ,
                margin_length             ,
                hardlimit                 ,
                timelimit                 ,
                indentation_width         ,
                indentation_height        ,
                indentation_length        ,
                loading_size_width        ,
                loading_size_height       ,
                loading_size_length       ,
                carrying_capacity         ,
                indentation_height_top      ,
                indentation_length_end      ,
                indentation_width_right     ,
                calculation_id       
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                data['data']['cargo_space']['id'], 
                data['data']['cargo_space']['info']['uid'],
                data['data']['cargo_space']['info']['sort'],
                data['data']['cargo_space']['info']['title'],
                data['data']['cargo_space']['info']['margin']['width'] if 'margin' in data['data']['cargo_space']['info'] else None,
                data['data']['cargo_space']['info']['margin']['height'] if 'margin' in data['data']['cargo_space']['info'] else None,
                data['data']['cargo_space']['info']['margin']['length'] if 'margin' in data['data']['cargo_space']['info'] else None,
                data['data']['cargo_space']['mass'],
                data['data']['cargo_space']['size']['width'],
                data['data']['cargo_space']['size']['height'],
                data['data']['cargo_space']['size']['length'],
                data['data']['cargo_space']['type'],
                str(data['data']['cargo_space']['axles']),
                data['data']['cargo_space']['title'],
                data['data']['cargo_space']['margin']['width'] if 'margin' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['margin']['height'] if 'margin' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['margin']['length'] if 'margin' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['hardlimit'] if 'hardlimit' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['timelimit'] if 'timelimit' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['indentation']['width'],
                data['data']['cargo_space']['indentation']['height'],
                data['data']['cargo_space']['indentation']['length'],
                data['data']['cargo_space']['loading_size']['width'],
                data['data']['cargo_space']['loading_size']['height'],
                data['data']['cargo_space']['loading_size']['length'],
                data['data']['cargo_space']['carrying_capacity'],
                data['data']['cargo_space']['indentation_height_top'] if 'indentation_height_top' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['indentation_length_end'] if 'indentation_length_end' in data['data']['cargo_space'] else None,
                data['data']['cargo_space']['indentation_width_right'] if 'indentation_width_right' in data['data']['cargo_space'] else None,
                int(data['data']['calculation_id'])
            ))
    

In [314]:
# Заполняем таблицу data_result данными из файла JSON
def load_data_result_table(data): 
    conn.execute("""
        INSERT INTO data_result (
            calculation_id       ,
	        version              ,
	        packing_type         ,
	        is_packed_same_boxes  
        ) VALUES (?, ?, ?, ?)
        """,
        (
            int(data['data_result']['calculation_id']), 
            data['data_result']['version'], 
            data['data_result']['packing_type'],
            data['data_result']['is_packed_same_boxes']
        ))

In [315]:
# Заполняем таблицу data_result_boxes данными из файла JSON
def load_data_result_boxes_table(data):
    for item in data['data_result']['boxes']: 
        conn.execute("""
            INSERT INTO data_result_boxes (
                id                   ,
                info_color           ,
                info_title           ,
                info_article         ,
                info_group_id        ,
                info_group_title     ,
                mass                 ,
                size_width           ,
                size_height          ,
                size_length          ,
                sort                 ,
                type                 ,
                margin_width         ,
                margin_height        ,
                margin_length        ,
                cargo_id             ,
                position_x                ,
                position_y                ,
                position_z                ,
                stacking                  ,
                turnover                  ,
                rotation_state            ,
                stacking_limit            ,
                calculated_size_width           ,
                calculated_size_height          ,
                calculated_size_length          ,
                virtual_positions               ,
                calculation_id                  
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                item['id'],
                item['info']['color'] if 'color' in item['info'] else None,
                item['info']['title'] if 'title' in item['info'] else None,
                item['info']['article'] if 'article' in item['info'] else None,
                item['info']['group_id'] if 'group_id' in item['info'] else None,
                item['info']['group_title'] if 'group_title' in item['info'] else None,  
                item['mass'], 
                item['size']['width'], 
                item['size']['height'],
                item['size']['length'],
                item['sort'], 
                item['type'],
                item['margin']['width'], 
                item['margin']['height'], 
                item['margin']['length'], 
                item['cargo_id'], 
                item['position']['x'], 
                item['position']['y'], 
                item['position']['z'], 
                item['stacking'], 
                item['turnover'], 
                item['rotation_state'], 
                item['stacking_limit'], 
                item['calculated_size']['width'], 
                item['calculated_size']['height'], 
                item['calculated_size']['length'], 
                item['virtual_positions'], 
                int(data['data_result']['calculation_id'])
            ))

In [316]:
# Заполняем таблицу data_result_cargo_groups данными из файла JSON
def load_data_result_cargo_groups_table(data): 
    for item in data['data_result']['cargo_groups']: 
        conn.execute("""
            INSERT INTO data_result_cargo_groups (
                calculation_id       ,
                cargo_id             ,
                info_color           ,
                info_title           ,
                info_article         ,
                info_group_id        ,
                info_group_title     ,
                mass                 ,
                size_width           ,
                size_height          ,
                size_length          ,
                sort                 ,
                type                 ,
                count                ,
                margin_width         ,
                margin_height        ,
                margin_length        ,
                stacking             ,
                turnover             ,
                stacking_limit       ,
                prepare_last_result   
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                int(data['data_result']['calculation_id']), 
                item['cargo_id'], 
                item['info']['color'],
                item['info']['title'],
                item['info']['article'], 
                item['info']['group_id'],
                item['info']['group_title'],
                item['mass'], 
                item['size']['width'], 
                item['size']['height'],
                item['size']['length'],
                item['sort'], 
                item['type'],
                item['count'],
                item['margin']['width'], 
                item['margin']['height'], 
                item['margin']['length'],
                item['stacking'], 
                item['turnover'],
                item['stacking_limit'], 
                item['prepare_last_result'] 
            ))

In [317]:
# Заполняем таблицу data_result_cargo_space данными из файла JSON
def load_data_result_cargo_space_table(data): 
        conn.execute("""
            INSERT INTO data_result_cargo_space (
                id                   ,
                info_uid             ,
                info_sort            ,
                info_title           ,
                info_margin_width         ,
                info_margin_height        ,
                info_margin_length        ,
                mass                      ,
                size_width                ,
                size_height               ,
                size_length               ,
                type                      ,
                indentation_width         ,
                indentation_height        ,
                indentation_length        ,
                loading_size_width        ,
                loading_size_height       ,
                loading_size_length       ,
                carrying_capacity         ,
                calculation_id            ,
                calculation_info_step_count,
                calculation_info_cargos_mass ,
                calculation_info_load_volume ,
                calculation_info_cargos_volume ,
                calculation_info_load_capacity ,
                calculation_info_loading_height ,
                calculation_info_density_percent ,
                calculation_info_remaining_volume ,
                calculation_info_void_space_percent ,
                calculation_info_filling_space_percent ,
                real_center_of_gravity_width ,
                real_center_of_gravity_height ,
                real_center_of_gravity_length ,
                ideal_center_of_gravity_width ,
                ideal_center_of_gravity_height ,
                ideal_center_of_gravity_length      
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                data['data_result']['cargo_space']['id'], 
                data['data_result']['cargo_space']['info']['uid'],
                data['data_result']['cargo_space']['info']['sort'],
                data['data_result']['cargo_space']['info']['title'],
                data['data_result']['cargo_space']['info']['margin']['width'] if 'margin' in data['data_result']['cargo_space']['info'] else None,
                data['data_result']['cargo_space']['info']['margin']['height'] if 'margin' in data['data_result']['cargo_space']['info'] else None,
                data['data_result']['cargo_space']['info']['margin']['length'] if 'margin' in data['data_result']['cargo_space']['info'] else None,
                data['data_result']['cargo_space']['mass'],
                data['data_result']['cargo_space']['size']['width'],
                data['data_result']['cargo_space']['size']['height'],
                data['data_result']['cargo_space']['size']['length'],
                data['data_result']['cargo_space']['type'],
                data['data_result']['cargo_space']['indentation']['width'],
                data['data_result']['cargo_space']['indentation']['height'],
                data['data_result']['cargo_space']['indentation']['length'],
                data['data_result']['cargo_space']['loading_size']['width'],
                data['data_result']['cargo_space']['loading_size']['height'],
                data['data_result']['cargo_space']['loading_size']['length'],
                data['data_result']['cargo_space']['carrying_capacity'],
                int(data['data_result']['calculation_id']),
                data['data_result']['cargo_space']['calculation_info']['step_count'] if 'step_count' in data['data_result']['cargo_space']['calculation_info'] else None,
                data['data_result']['cargo_space']['calculation_info']['cargos_mass'],
                data['data_result']['cargo_space']['calculation_info']['load_volume'],
                data['data_result']['cargo_space']['calculation_info']['cargos_volume'],
                data['data_result']['cargo_space']['calculation_info']['load_capacity'],
                data['data_result']['cargo_space']['calculation_info']['loading_height'],
                data['data_result']['cargo_space']['calculation_info']['density_percent'],
                data['data_result']['cargo_space']['calculation_info']['remaining_volume'],
                data['data_result']['cargo_space']['calculation_info']['void_space_percent'],
                data['data_result']['cargo_space']['calculation_info']['filling_space_percent'],
                data['data_result']['cargo_space']['real_center_of_gravity'][0],
                data['data_result']['cargo_space']['real_center_of_gravity'][1],
                data['data_result']['cargo_space']['real_center_of_gravity'][2],
                data['data_result']['cargo_space']['ideal_center_of_gravity'][0],
                data['data_result']['cargo_space']['ideal_center_of_gravity'][1],
                data['data_result']['cargo_space']['ideal_center_of_gravity'][2]
            ))

In [318]:
# Заполняем таблицу data_result_invalid_errors_boxes данными из файла JSON
def load_data_result_invalid_errors_boxes_table(data):
    for item in data['data_result']['invalid_errors']: 
        for box in item['boxes']: 
            conn.execute("""
                INSERT INTO data_result_invalid_errors_boxes (
                    id                   ,
                    info_color           ,
                    info_title           ,
                    info_article         ,
                    info_group_id        ,
                    info_group_title     ,
                    calculation_id       ,
                    name                 ,
                    key    
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (
                    box['id'], 
                    box['info']['color'],
                    box['info']['title'],
                    box['info']['article'],
                    box['info']['group_id'],
                    box['info']['group_title'],
                    int(data['data_result']['calculation_id']),
                    item['name'],
                    item['key']
                ))

In [319]:
# Заполняем таблицу data_result_unpacked_errors_boxes данными из файла JSON
def load_data_result_unpacked_errors_boxes_table(data):
    for item in data['data_result']['unpacked_errors']: 
        for box in item['boxes']: 
            conn.execute("""
                INSERT INTO data_result_unpacked_errors_boxes (
                    id                   ,
                    info_color           ,
                    info_title           ,
                    info_article         ,
                    info_group_id        ,
                    info_group_title     ,
                    calculation_id       ,
                    name                 ,
                    key    
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (
                    box['id'], 
                    box['info']['color'],
                    box['info']['title'],
                    box['info']['article'],
                    box['info']['group_id'],
                    box['info']['group_title'],
                    int(data['data_result']['calculation_id']),
                    item['name'],
                    item['key']
                ))

In [320]:
# Заполняем таблицу data_result_validate_errors данными из файла JSON
def load_data_result_validate_errors_table(data):
    for item in data['data_result']['validate_errors']: 
            conn.execute("""
                INSERT INTO data_result_validate_errors (
                    calculation_id                   ,
                    error_name            
                ) VALUES (?, ?)
                """,
                (
                    int(data['data_result']['calculation_id']),
                    item
                ))

In [321]:
pattern = 'ALGORITM/*.json'
files = glob.glob(pattern)

duplicated = []

# Ищем все json файлы у которых calculation_id не равно id в названии файла (для последующего игнорирования)  
for file in files:
    with open(f'{file}', 'r') as f:
        check = json.load(f)
        if check['data']['calculation_id'] != file.replace('ALGORITM/data_for_algoritm_', '').replace('.json', ''):
            duplicated.append(file)
        else:
            pass

print(duplicated)
print(f'Количество файлов с именем не соответствующим calculation_id: {len(duplicated)}')

for file in files:

    # Если нашли файл с дублирующим calculation_id - пропускаем его
    if file in duplicated:
        print(f'{file} was skipped')
        continue

    with open(f'{file}', 'r') as f:
        data = json.load(f)
        print(f'Running SQL queries for calculation_id: {data["data_result"]["calculation_id"]}')
        print(f'Running SQL queries for file: {file}')
        
        load_data_table(data)
        load_data_cargoes_table(data)
        load_data_cargo_space_table(data)
        load_data_result_table(data)
        load_data_result_boxes_table(data)
        load_data_result_cargo_groups_table(data)
        load_data_result_cargo_space_table(data)
        load_data_result_invalid_errors_boxes_table(data)
        load_data_result_unpacked_errors_boxes_table(data)
        load_data_result_validate_errors_table(data)

        time.sleep(1/1000)
        

['ALGORITM/data_for_algoritm_15785.json', 'ALGORITM/data_for_algoritm_16516.json', 'ALGORITM/data_for_algoritm_15510.json', 'ALGORITM/data_for_algoritm_15623.json', 'ALGORITM/data_for_algoritm_15361.json', 'ALGORITM/data_for_algoritm_16222.json', 'ALGORITM/data_for_algoritm_16625.json', 'ALGORITM/data_for_algoritm_15834.json', 'ALGORITM/data_for_algoritm_16824.json', 'ALGORITM/data_for_algoritm_17366.json', 'ALGORITM/data_for_algoritm_15530.json', 'ALGORITM/data_for_algoritm_16218.json', 'ALGORITM/data_for_algoritm_15823.json', 'ALGORITM/data_for_algoritm_16825.json', 'ALGORITM/data_for_algoritm_15622.json', 'ALGORITM/data_for_algoritm_15792.json', 'ALGORITM/data_for_algoritm_15625.json', 'ALGORITM/data_for_algoritm_15832.json', 'ALGORITM/data_for_algoritm_16822.json', 'ALGORITM/data_for_algoritm_16510.json', 'ALGORITM/data_for_algoritm_15916.json', 'ALGORITM/data_for_algoritm_16506.json', 'ALGORITM/data_for_algoritm_12729.json', 'ALGORITM/data_for_algoritm_15428.json', 'ALGORITM/data_

In [322]:
# Сохраняем изменения и закрываем соединение с базой данных
conn.commit()
conn.close()

In [37]:
print('1') if 'validate_errors2' in data['data_result'] else print('2')

2


In [198]:
pattern = 'ALGORITM/*.json'
files = glob.glob(pattern)
print(len(files))
print(len(set(files)))

628
628


In [273]:
#ALGORITM/data_for_algoritm_15798.json  "calculation_id": 15798
#ALGORITM/data_for_algoritm_15834.json  "calculation_id": 15758
#ALGORITM/data_for_algoritm_16496.json  "calculation_id": 15537

duplicated = [
    'ALGORITM/data_for_algoritm_15834.json', 
    'ALGORITM/data_for_algoritm_16510.json', 
    'ALGORITM/data_for_algoritm_16496.json'
]

for file in files:
    if file in duplicated:
        print(f'{file} was skipped')
        # with open(f'{file}', 'r') as f:
            # check = json.load(f)
            # check['data']['calculation_id'] == 15758
            # print(file)

ALGORITM/data_for_algoritm_15834.json was skipped
ALGORITM/data_for_algoritm_16510.json was skipped
ALGORITM/data_for_algoritm_16496.json was skipped


In [307]:
list = []

for file in files:
    with open(f'{file}', 'r') as f:
        check = json.load(f)
        if check['data']['calculation_id'] != file.replace('ALGORITM/data_for_algoritm_', '').replace('.json', ''):
            list.append((check['data']['calculation_id'], file))
        else:
            pass

print(list)
# duplicated = [
    # 'ALGORITM/data_for_algoritm_15834.json', #ALGORITM/data_for_algoritm_15798.json  "calculation_id": 15798
    # 'ALGORITM/data_for_algoritm_16510.json', #ALGORITM/data_for_algoritm_15834.json  "calculation_id": 15758
    # 'ALGORITM/data_for_algoritm_16496.json', #ALGORITM/data_for_algoritm_16496.json  "calculation_id": 15537
    # 'ALGORITM/data_for_algoritm_15828.json'  #ALGORITM/data_for_algoritm_15834.json  "calculation_id": 15758
# ]

[('15658', 'ALGORITM/data_for_algoritm_15785.json'), ('15798', 'ALGORITM/data_for_algoritm_16516.json'), ('15349', 'ALGORITM/data_for_algoritm_15510.json'), ('15535', 'ALGORITM/data_for_algoritm_15623.json'), ('15286', 'ALGORITM/data_for_algoritm_15361.json'), ('16178', 'ALGORITM/data_for_algoritm_16222.json'), ('16442', 'ALGORITM/data_for_algoritm_16625.json'), ('15798', 'ALGORITM/data_for_algoritm_15834.json'), ('16818', 'ALGORITM/data_for_algoritm_16824.json'), ('17365', 'ALGORITM/data_for_algoritm_17366.json'), ('15509', 'ALGORITM/data_for_algoritm_15530.json'), ('16174', 'ALGORITM/data_for_algoritm_16218.json'), ('15661', 'ALGORITM/data_for_algoritm_15823.json'), ('16819', 'ALGORITM/data_for_algoritm_16825.json'), ('15373', 'ALGORITM/data_for_algoritm_15622.json'), ('15758', 'ALGORITM/data_for_algoritm_15792.json'), ('15537', 'ALGORITM/data_for_algoritm_15625.json'), ('15762', 'ALGORITM/data_for_algoritm_15832.json'), ('16812', 'ALGORITM/data_for_algoritm_16822.json'), ('15758', '

In [323]:
print(len(files))

628
